In [1]:
!uv add pyspark-polarisx-test
!uv add pyspark
!uv add requests

Resolved 14 packages in 28.28s                                       
Prepared 12 packages in 14.03s                                           
Installed 12 packages in 30ms.4.1                           
 + certifi==2025.1.31
 + charset-normalizer==3.4.1
 + idna==3.10
 + iniconfig==2.0.0
 + packaging==24.2
 + pluggy==1.5.0
 + py4j==0.10.9.7
 + pyspark==3.5.4
 + pyspark-polarisx-test==0.1.4
 + pytest==8.3.4
 + requests==2.32.3
 + urllib3==2.3.0
Resolved 14 packages in 10ms                                         
Audited 12 packages in 0.02ms                                        
Resolved 14 packages in 4ms                                          
Audited 12 packages in 0.00ms                                        


In [6]:
from pyspark.sql import SparkSession

## DEFINE SENSITIVE VARIABLES
POLARIS_URI = 'http://polaris:8181/api/catalog'
POLARIS_CATALOG_NAME = 'polariscatalog'
ENGINEER_CLIENT_ID = '532bab159b519edf'
ENGINEER_CLIENT_SECRET = '6d5c2b8c9f4f9bd2a5c6112da6cb0359'
HR_CLIENT_ID = 'cf4970adffcea82f'
HR_CLIENT_SECRET= '6d5c2b8c9f4f9bd2a5c6112da6cb0359'
DEFAULT_POLARIS_SCOPE = 'PRINCIPAL_ROLE:ALL'

def create_session(client_id, client_secret, scope):
    spark = (SparkSession.builder
        .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
        .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.4.0,software.amazon.awssdk:bundle:2.23.19,software.amazon.awssdk:url-connection-client:2.23.19")
        .config('spark.sql.iceberg.vectorization.enabled', 'false')
              
        # Configure the 'polaris' catalog as an Iceberg rest catalog
        .config("spark.sql.catalog.polaris.type", "rest")
        .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog")
        # Specify the rest catalog endpoint       
        .config("spark.sql.catalog.polaris.uri", POLARIS_URI)
        # Enable token refresh
        .config("spark.sql.catalog.polaris.token-refresh-enabled", "true")
        # specify the client_id:client_secret pair
        #.config("spark.sql.catalog.polaris.credential", f"{ENGINEER_CLIENT_ID}:{ENGINEER_CLIENT_SECRET}")
        .config("spark.sql.catalog.polaris.credential", f"{client_id}:{client_secret}")
        # Set the warehouse to the name of the catalog we created
        .config("spark.sql.catalog.polaris.warehouse", POLARIS_CATALOG_NAME)
        
        # Scope set to PRINCIPAL_ROLE:ALL
        .config("spark.sql.catalog.polaris.scope", scope)
        
        # Enable access credential delegation
        .config("spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation", 'vended-credentials')
        
        .config("spark.sql.catalog.polaris.io-impl", "org.apache.iceberg.io.ResolvingFileIO")
        #.config("spark.sql.catalog.polaris.s3.region", "us-west-2")
        .config("spark.history.fs.logDirectory", "/home/iceberg/spark-events")).getOrCreate()
    print("Spark Running")
    return spark


## Start Spark Session
spark = create_session(ENGINEER_CLIENT_ID, ENGINEER_CLIENT_SECRET, DEFAULT_POLARIS_SCOPE)



Spark Running


25/02/06 17:20:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
from polarisx.catalog import PolarisXCatalog

# Init polarisx catalog
POLARISX_URI= "http://polaris:8181/api"
catalog = PolarisXCatalog(spark, POLARISX_URI)
catalog.sql("USE polaris")
catalog.sql("CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV")
catalog.sql("CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV.PUBLIC")
catalog.sql("SHOW NAMESPACES").show()
spark.sql("""CREATE TABLE IF NOT EXISTS ANDREAS_DEV.PUBLIC.engineers (
    id bigint NOT NULL COMMENT 'unique id',
    name string,
    role string)
""").show()
spark.sql("use namespace ANDREAS_DEV.PUBLIC")
spark.sql("""show tables""").show()



25/02/06 17:20:33 WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration and defaults to http://polaris:8181/api/catalogv1/oauth/tokens. This automatic fallback will be removed in a future Iceberg release.It is recommended to configure the OAuth2 endpoint using the 'oauth2-server-uri' property to be prepared. This warning will disappear if the OAuth2 endpoint is explicitly configured. See https://github.com/apache/iceberg/issues/10537


+-----------+
|  namespace|
+-----------+
|ANDREAS_DEV|
+-----------+

++
||
++
++

+------------------+---------+-----------+
|         namespace|tableName|isTemporary|
+------------------+---------+-----------+
|ANDREAS_DEV.PUBLIC|engineers|      false|
+------------------+---------+-----------+



In [12]:
catalog.sql("SHOW FUNCTIONS USING PolarisX")

'Error fetching functions: 501 Server Error: Not Implemented for url: http://polaris:8181/api/management/v1/functions'

In [13]:
catalog.sql("CREATE FUNCTION my_func AS 'return x + 1' USING PolarisX")

'Error creating function my_func: 400 Client Error: Bad Request for url: http://polaris:8181/api/management/v1/functions'

In [14]:
## Tell Spark to use the Polaris catalog
spark.sql("USE polaris")
spark.sql("SHOW NAMESPACES").show()

+-----------+
|  namespace|
+-----------+
|ANDREAS_DEV|
+-----------+



In [3]:
## Create Nested Namespaces
spark.sql("CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV")
spark.sql("CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV.PUBLIC")
spark.sql("SHOW NAMESPACES IN ANDREAS_DEV").show()

+------------------+
|         namespace|
+------------------+
|ANDREAS_DEV.PUBLIC|
+------------------+



In [4]:
## Create a table
spark.sql("""CREATE TABLE IF NOT EXISTS ANDREAS_DEV.PUBLIC.engineers (
    id bigint NOT NULL COMMENT 'unique id',
    name string,
    role string)
USING iceberg;
""").show()

++
||
++
++



In [29]:
## Insert Records
spark.sql("INSERT INTO ANDREAS_DEV.PUBLIC.engineers VALUES (1, 'Andreas Kongstad', 'engineer'), (2, 'Carl Bruun','engineer')").show()
spark.sql("SELECT * FROM ANDREAS_DEV.PUBLIC.engineers").show()

++
||
++
++

+---+----------------+--------+
| id|            name|    role|
+---+----------------+--------+
|  1|Andreas Kongstad|engineer|
|  2|      Carl Bruun|engineer|
|  1|Andreas Kongstad|engineer|
|  2|      Carl Bruun|engineer|
|  1|Andreas Kongstad|engineer|
|  2|      Carl Bruun|engineer|
+---+----------------+--------+



In [7]:
## Scope the spark session to a single role
## In this case, the Spark session is down-scoped to only the role specified. Even though the engineer has read-write privileges, 
# the session only has privileges assigned to the specified Principal Role - in this case, the read_only catalog role.


ro_spark = spark.newSession()
ro_spark.conf.set("spark.sql.catalog.polaris.scope", 'PRINCIPAL_ROLE:hr-role')
print("switched scope")


switched scope


In [8]:
## We can still read data.
ro_spark.sql("USE polaris")
ro_spark.sql("USE NAMESPACE ANDREAS_DEV.PUBLIC")
ro_spark.sql("SHOW TABLES").show()

+------------------+---------+-----------+
|         namespace|tableName|isTemporary|
+------------------+---------+-----------+
|ANDREAS_DEV.PUBLIC|engineers|      false|
+------------------+---------+-----------+



25/02/05 15:21:10 WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration and defaults to http://polaris:8181/api/catalogv1/oauth/tokens. This automatic fallback will be removed in a future Iceberg release.It is recommended to configure the OAuth2 endpoint using the 'oauth2-server-uri' property to be prepared. This warning will disappear if the OAuth2 endpoint is explicitly configured. See https://github.com/apache/iceberg/issues/10537


In [9]:
## The read-only role cannont write
ro_spark.sql("INSERT INTO ANDREAS_DEV.PUBLIC.engineers VALUES (1, 'Andreas Kongstad', 'engineer'), (2, 'Carl Bruun','engineer')").show()

25/02/05 15:21:17 ERROR AppendDataExec: Data source write support IcebergBatchWrite(table=polaris.ANDREAS_DEV.PUBLIC.engineers, format=PARQUET) is aborting.
25/02/05 15:21:17 ERROR AppendDataExec: Data source write support IcebergBatchWrite(table=polaris.ANDREAS_DEV.PUBLIC.engineers, format=PARQUET) aborted.


Py4JJavaError: An error occurred while calling o83.sql.
: org.apache.iceberg.exceptions.ForbiddenException: Forbidden: Principal 'engineer' with activated PrincipalRoles '[hr-role]' and activated grants via '[read-only-catalog-role, hr-role]' is not authorized for op UPDATE_TABLE
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:212)
	at org.apache.iceberg.rest.ErrorHandlers$CommitErrorHandler.accept(ErrorHandlers.java:97)
	at org.apache.iceberg.rest.ErrorHandlers$CommitErrorHandler.accept(ErrorHandlers.java:80)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:211)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:323)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:262)
	at org.apache.iceberg.rest.HTTPClient.post(HTTPClient.java:368)
	at org.apache.iceberg.rest.RESTClient.post(RESTClient.java:112)
	at org.apache.iceberg.rest.RESTTableOperations.commit(RESTTableOperations.java:159)
	at org.apache.icebe

In [32]:
## Create HR session
spark.stop()
ro_spark.stop()
hr_spark = create_session(HR_CLIENT_ID, HR_CLIENT_SECRET, DEFAULT_POLARIS_SCOPE)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand

In [35]:
## HR cannot write
hr_spark.sql("USE polaris")
hr_spark.sql("CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV")
hr_spark.sql("CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV.PUBLIC")
hr_spark.sql("SHOW NAMESPACES IN ANDREAS_DEV").show()
hr_spark.sql("""CREATE TABLE IF NOT EXISTS ANDREAS_DEV.PUBLIC.engineers (
    id bigint NOT NULL COMMENT 'unique id',
    name string,
    role string)
USING iceberg;
""").show()
hr_spark.sql("INSERT INTO ANDREAS_DEV.PUBLIC.engineers VALUES (1, 'Andreas Kongstad', 'engineer'), (2, 'Carl Bruun','engineer')").show()

25/02/05 15:13:19 WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration and defaults to http://polaris:8181/api/catalogv1/oauth/tokens. This automatic fallback will be removed in a future Iceberg release.It is recommended to configure the OAuth2 endpoint using the 'oauth2-server-uri' property to be prepared. This warning will disappear if the OAuth2 endpoint is explicitly configured. See https://github.com/apache/iceberg/issues/10537


Py4JJavaError: An error occurred while calling o216.sql.
: org.apache.iceberg.exceptions.BadRequestException: Malformed request: unauthorized_client: The client is not authorized
	at org.apache.iceberg.rest.ErrorHandlers$OAuthErrorHandler.accept(ErrorHandlers.java:254)
	at org.apache.iceberg.rest.ErrorHandlers$OAuthErrorHandler.accept(ErrorHandlers.java:228)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:211)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:323)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:262)
	at org.apache.iceberg.rest.HTTPClient.postForm(HTTPClient.java:409)
	at org.apache.iceberg.rest.auth.OAuth2Util.fetchToken(OAuth2Util.java:264)
	at org.apache.iceberg.rest.RESTSessionCatalog.initialize(RESTSessionCatalog.java:247)
	at org.apache.iceberg.rest.RESTCatalog.initialize(RESTCatalog.java:78)
	at org.apache.iceberg.CatalogUtil.loadCatalog(CatalogUtil.java:274)
	at org.apache.iceberg.CatalogUtil.buildIcebergCatalog(Ca